# Sentiment Analysis

What is the overall sentiment of tweets pertaining to covid? How does that change according to province/over time? Are tweets talking about different topics specific pain points?

In [6]:
from utils import DTYPE, PARSE_DATES, PROV_CONSOLIDATION, CONSOLIDATED_PROVINCES, CONVERTERS, ANCHOR_NAMES, PROVINCE_COLOR_MAP
from tqdm.auto import tqdm
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import glob
tqdm.pandas()


# prov_map = lambda x : x if x not in PROV_CONSOLIDATION else PROV_CONSOLIDATION[x]

# total_df = pd.read_csv("../data/processed_data/total_tweet_dataset.csv",header=0,dtype=DTYPE,converters=CONVERTERS,parse_dates=PARSE_DATES)
# total_df = total_df.set_index("id").sort_values("created_at")[~total_df.index.duplicated()]

# total_df["created_at"] = total_df["created_at"].dt.to_period("D").dt.to_timestamp('s')
# total_df["province"] = total_df["province"].apply(prov_map)
# total_df = total_df[total_df.clean_text.notnull()]
# total_df["province"] = total_df["province"].apply(prov_map)
# total_df = total_df[total_df["province"].isin(CONSOLIDATED_PROVINCES)]
# print(len(total_df))
# total_df.head()

/Users/cameron/.local/share/virtualenvs/COVID-Child-Care-Twitter-OW82RhPk/lib/python3.7/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from kaleido.scopes.plotly import PlotlyScope
scope = PlotlyScope()
vis_args = {
    "template": "simple_white",
    "font":{"size": 23},
    "width": 1000
}

In [3]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
tb = Blobber()

engine = "Vader"
sentiment_engines = {
    "NaiveBayes": lambda x : tb(x).polarity,
    "Vader": lambda x : analyzer.polarity_scores(x)['compound']
}

total_df["polarity"] = total_df[["clean_text"]].progress_apply(lambda x: sentiment_engines[engine](x["clean_text"]),axis=1)
total_df["polarity_bucket"] = total_df["polarity"].map(lambda x : "positive" if x >= 0.1 else "negative" if x <= -0.1 else "NA")
total_df.head()


In [4]:
sent = pd.DataFrame()
sent["tb"] = total_df[["clean_text"]].progress_apply(lambda x: sentiment_engines["NaiveBayes"](x["clean_text"]),axis=1)
sent["vader"] = total_df[["clean_text"]].progress_apply(lambda x: sentiment_engines["Vader"](x["clean_text"]),axis=1)

In [14]:
args = {
    "x": "tb",
    "y": "vader",
    "template": "simple_white",
    "labels": {"tb": "Text Blob (Naive Bayes)", "vader": "Vader Sentiment Analyzer"},
    "opacity": 0.05,
    "trendline": "ols",
    "trendline_color_override": "red"
}
fig = px.scatter(sent, **args)
px.get_trendline_results(fig).px_fit_results.iloc[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                 1.235e+05
Date:                Wed, 28 Oct 2020   Prob (F-statistic):               0.00
Time:                        10:22:15   Log-Likelihood:            -2.3621e+05
No. Observations:              402217   AIC:                         4.724e+05
Df Residuals:                  402215   BIC:                         4.724e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1051      0.001    141.471      0.000       0.104       0.107
x1             1.0064      0.003    351.366      0.000       1.001       1.012
==============================================================================
Omnibus:                    15123.551   Durbin-Watson:                   1.514
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16003.196
Skew:                          -0.468   Prob(JB):                         0.00
Kurtosis:                       2.718   Cond. No.                         4.22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
fig.update_layout(**vis_args)

fp = f"../visualizations/sentiment_analysis/vader_tb-comp"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))

In [59]:
sent["residual"] = sent["tb"]-sent["vader"]
args = {
    "x": "residual",
    "labels": {"residual": "Residual","count": "Frequency"},
    "nbins":20
}
fig = px.histogram(sent, **args)
fig.update_layout(**vis_args)
fp = f"../visualizations/sentiment_analysis/vader_tb-residual_hist"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))

In [ ]:
c = total_df[["polarity_bucket","source"]].groupby("polarity_bucket").count().rename({"source":"count"},axis=1).reset_index()
fig = px.pie(c, values='count', names='polarity_bucket')
fig.show()


In [ ]:
by_date = total_df.groupby(["created_at","polarity_bucket"]).count()[["source"]].rename({"source":"count"},axis=1).reset_index()
by_date["percentage"] = by_date['count'] / by_date.groupby('created_at')['count'].transform('sum')
args = {
    "x": "created_at",
    "y": "percentage",
    "color": "polarity_bucket",
    "template": "simple_white",
    "labels": {"percentage": "Proportion of Tweets", "created_at": "Date", "polarity_bucket": "Sentiment"}
}
fig = px.area(by_date,**args)
fig.update_yaxes(dict(tickformat=',.0%'))
fig.update_layout(**vis_args)
fp = f"../visualizations/sentiment_analysis/{engine}-sentiment_proportion"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()

In [ ]:

by_date = total_df.groupby([pd.Grouper(key='created_at', freq='W-MON'),"polarity_bucket"]).count()[["source"]].rename({"source":"count"},axis=1).reset_index()
by_date["percentage"] = by_date['count'] / by_date.groupby(pd.Grouper(key='created_at', freq='W-MON'))['count'].transform('sum')
args = {
    "x": "created_at",
    "y": "percentage",
    "color": "polarity_bucket",
    "template": "simple_white",
    "labels": {"percentage": "Proportion of Tweets", "created_at": "Date", "polarity_bucket": "Sentiment"}
}
fig = px.area(by_date,**args)
fig.update_yaxes(dict(tickformat=',.0%'))
fig.update_layout(**vis_args)
fp = f"../visualizations/sentiment_analysis/{engine}-sentiment_proportion"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()

In [ ]:
from sklearn.linear_model import LinearRegression

def r2(x,y):
    x = x.to_numpy(dtype=float).reshape(-1,1)
    y = y.to_numpy(dtype=float)
    # Fit a linear regression to our model and its results
    reg = LinearRegression().fit(x, y) 
    return reg.score(x, y)

for bucket in by_date["polarity_bucket"].unique():
    iso = by_date[by_date["polarity_bucket"] == bucket]
    r = r2(iso.index,iso["count"])
    print(f"{bucket} r^2={r:.4f}")

In [ ]:
list(total_df.sort_values("polarity",ascending=False).head()["original_text"])

## Sentiment by Topic



In [ ]:
sentiment_avg = total_df.groupby([pd.Grouper(key='created_at', freq='W-MON'),"cluster_name"])[["polarity"]].mean().rename({"polarity":"polarity_mean"},axis=1).reset_index()
args = {
    "x": "created_at",
    "y": "polarity_mean",
    "color": "cluster_name",
    "template": "simple_white",
    "labels": {"polarity_mean": "Average Sentiment", "created_at": "Date", "cluster_name": "Topic"},
    "line_shape": "spline"
}
fig = px.line(sentiment_avg,**args)
fig.update_layout(**vis_args)
fp = f"../visualizations/sentiment_analysis/{engine}-topic_timeseries"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()

In [ ]:
topic_sent_corr = sentiment_avg.pivot(index="created_at",values="polarity_mean",columns="cluster_name").corr()
corr_matrix = topic_sent_corr.to_numpy()
labels = topic_sent_corr.columns
for i in range(len(corr_matrix)):
    corr_matrix[i,i] = None
    


data = go.Heatmap(z=corr_matrix,y=labels,x=labels)
fig = go.Figure(data=data,layout=vis_args)

fp = f"../visualizations/sentiment_analysis/{engine}-topic_sentiment_correlation"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()

In [ ]:
fig = go.Figure()

for prov in CONSOLIDATED_PROVINCES:
    iso = total_df[total_df["province"]==prov]
    fig.add_trace(go.Box(
        y=iso["polarity"],
        x=iso["cluster_name"],
#         boxpoints=False,
        name=prov,
        marker_color=PROVINCE_COLOR_MAP[prov]
    ))
    
fig.add_trace(go.Box(
        y=total_df["polarity"],
        x=total_df["cluster_name"],
#         boxpoints=False,
        name='Total',
        marker_color=PROVINCE_COLOR_MAP['Total']
    ))

args = {
    "font":{"size": 23},
    "height": 700,
    "width": 2000,    
    "template":"simple_white",
    "yaxis":{"showline": False},
    "boxmode":'group' # group together boxes of the different traces for each value of x

}
fig.update_layout(**args)
fp = f"../visualizations/sentiment_analysis/{engine}-box_plot-sentiment_by_province"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))


## Sentiment by Province

In [ ]:
sentiment_avg = total_df.groupby(["province","cluster_name"])[["polarity"]].mean().rename({"polarity":"polarity_mean"},axis=1).reset_index()
data = []
for name in ANCHOR_NAMES+["Overflow"]:
    iso = sentiment_avg[sentiment_avg["cluster_name"]==name]
    data.append(go.Bar(name=name, x=iso["province"], y=iso["polarity_mean"]))
fig = go.Figure(data=data)
args = {
    "font":{"size": 23},
    "height": 700,
    "template":"simple_white",
    "yaxis":{"showline": False},
    "xaxis":{"showline": False},
    "boxmode":'group' # group together boxes of the different traces for each value of x

}
fig.update_layout(**{**args,**vis_args})
fp = f"../visualizations/sentiment_analysis/{engine}-sentiment_by_province"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()


In [ ]:
sentiment_avg = total_df.groupby([pd.Grouper(key='created_at', freq='W-MON'),"province"])[["polarity"]].mean().rename({"polarity":"polarity_mean"},axis=1).reset_index()
args = {
    "x": "created_at",
    "y": "polarity_mean",
    "color": "province",
    "template": "simple_white",
    "labels": {"polarity_mean": "Average Sentiment", "created_at": "Date", "province": "Province"},
    "line_shape": "spline",
    "color_discrete_map": PROVINCE_COLOR_MAP
}
fig = px.line(sentiment_avg,**args)
fig.update_layout(**vis_args)
fp = f"../visualizations/sentiment_analysis/{engine}-province_timeseries"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()

In [ ]:
prov_sent_corr = sentiment_avg.pivot(index="created_at",values="polarity_mean",columns="province").corr()
prov_sent_corr.to_excel("../data/province_sentiment-correlation.xlsx")
corr_matrix = prov_sent_corr.to_numpy()
labels = prov_sent_corr.columns
for i in range(len(corr_matrix)):
    corr_matrix[i,i] = None

data = go.Heatmap(z=corr_matrix,y=labels,x=labels)
fig = go.Figure(data=data,layout=vis_args)

fp = f"../visualizations/sentiment_analysis/{engine}-prov_sentiment_correlation"
with open(f"{fp}.pdf", "wb") as f:
    f.write(scope.transform(fig, format="pdf"))
fig.show()